In [1]:
from gym import Env
import numpy as np
from gym.spaces import MultiDiscrete,Box

In [2]:
#np.random.randint??

In [3]:
def simulatepandemic(observation,actions):
    #update observation
    return np.random.randint(0,5,size=[4,5])

def initializepandemic():
    #observation could be vector of all individuals with states e.g. [0,1,2,0,3,0,...]
    n = 100
    return np.random.randint(0,5,size=[4,5])

#write observation as n x states [[0,1,0,0,0],[1,0,0,0,0],...]

In [4]:
class PanEnv(Env):
    def __init__(self,n):#n number of people
        self.n = n
        self.observation = None
        self.action_space = MultiDiscrete(nvec=[10,10,10,10])
        '''multidiscrete mapping, NN will give us vector with values 0-10
        we need to convert this map into meaning for our action > e.g. nvec[0]/sum(nvec) describes relative 
        availability. length of nvec == number of age groups'''
        self.observation_space = Box(low=-np.inf,high=np.inf,shape=[4,5])
    def step(self,actions):
        observation = simulatepandemic(self.observation,actions)
        #observation (object): agent's observation of the current environment
        reward = np.sum(observation)
        #reward (float) : amount of reward returned after previous action
        #negative reward: punishment > change weights a lot, push away from causing weights, positive rewards pull
        #do reward compared to reward from previous step
        #naive example: reward = -sum(infected) > we want a reward where the cumulative sum of infections until end
        #is minimized
        #exp. solution: store information in self, summed infections, normalized by time
        done = np.all(observation>1)#pairwise AND
        #done (bool): whether the episode has ended, in which case further step() calls will return undefined results
        info = {}
        #info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)'''
        return observation, reward, done, info
    def reset(self):
        #returns initial state
        self.observation = initializepandemic()
        print(self.observation)
        return self.observation

In [5]:
env = PanEnv(n=100)
env.reset()

[[1 0 0 1 0]
 [2 2 0 1 2]
 [1 1 4 3 0]
 [4 4 3 0 2]]


array([[1, 0, 0, 1, 0],
       [2, 2, 0, 1, 2],
       [1, 1, 4, 3, 0],
       [4, 4, 3, 0, 2]])

In [6]:
actions = np.random.randint(1,5,100)

In [7]:
env.step(actions)

(array([[1, 0, 1, 4, 2],
        [0, 2, 0, 0, 0],
        [0, 2, 3, 3, 1],
        [2, 0, 2, 4, 0]]),
 27,
 False,
 {})

In [8]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
# Parallel environments

env = DummyVecEnv([lambda: PanEnv(n=100)])

model = PPO("MlpPolicy", env, verbose=1) #multilayer
model.learn(total_timesteps=25000) #training loop
#model.save("ppo_cartpole")

#del model # remove to demonstrate saving and loading

#model = PPO.load("ppo_cartpole")

RuntimeError: KeyboardInterrupt: 

In [ ]:
obs = env.reset()
while not done:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)

In [ ]:
#split into learning and testing
model.learn(total_timesteps = 5000)
# store/accumulate rewards